# DeepVerse-Python ISAC Demo

## Introduction

In this notebook, we present an ISAC demo where the radar signal at the base station is used to predict the optimal communication beam for the BS to serve the mobile user (vehicle), using a deep learning model.
- Data generation: How to generate radar and communication data using DeepVerse-Python
- Data postprocessing: How to prepare the training and test datasets
- Radar-aided beam prediction: Deep learning model training and testing


See more detailed [API documentation](https://deepverse6g.net/documentation)\
See more detailed [DeepVerse generator tutorial](https://deepverse6g.net/tutorials)


## Installation

In [ ]:
# # Installation needed starting from a fresh conda env
# # This may not produce a torch installation supporting cuda and gpu
# %pip install torch numpy pandas tqdm scikit-learn requests deepverse

## Import

In [5]:
import os
import sys

import torch
import torch.nn as nn
import torch.nn.functional as F

import zipfile
import requests
import shutil
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from deepverse import ParameterManager, Dataset # pip install deepverse

## Download DeepVerse data

Here we download the DeepVerse data and prepare the scenario folder. We use the DT1 scenario in this demo. The DT1 incoporates the LiDAR, RGB images, wireless, and position data modalities. Since this demo focuses on the ISAC application, we only download and use the wireless data modality.

The scenario folder follows the below structure
```
DeepVerse-main/
├─ scenarios/
│  ├─ DT1/
│  │  ├─ wireless/
│  │  │  ├─ ...
│  │  │  ├─ params.mat
│  │  ├─ param/
│  │  │  ├─ config.m
│  │  ├─ scenario1.csv
```

In [ ]:
# Set up directories
scenario_name = 'DT1'
scenario_dir = Path(f"scenarios/{scenario_name}")
scenario_dir.mkdir(parents=True, exist_ok=True)

def download_and_unzip(url, zip_path, extract_to):
    response = requests.get(url, stream=True)
    response.raise_for_status()
    
    total = int(response.headers.get('content-length', 0))
    with open(zip_path, 'wb') as f, tqdm(
        desc=f"Downloading: {zip_path.name}",
        total=total,
        unit='B',
        unit_scale=True,
        unit_divisor=1024,
    ) as bar:
        for chunk in response.iter_content(chunk_size=8192):
            size = f.write(chunk)
            bar.update(size)

    print(f"Infalting: {zip_path}")
    try:
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_to)
    except zipfile.BadZipFile:
        print(f"Error: {zip_path} is not a valid zip file!")
        return

    print(f"Removing zip: {zip_path}")
    zip_path.unlink()

# Download and extract wireless data
print("Preparing wireless data")
download_and_unzip(
    "https://www.dropbox.com/scl/fi/pyzmcb5jv3oo9e29nymw8/wireless.zip?rlkey=b5gbkgd8dng6hpf91rv7za9ix&e=1&st=5opqw9lc&dl=1",
    scenario_dir / "wireless.zip",
    scenario_dir
)

# Download and extract parameter files
print("Preparing param files")
param_dir = scenario_dir / "param"
param_dir.mkdir(exist_ok=True)
download_and_unzip(
    "https://www.dropbox.com/scl/fo/9sfd6u8912l7o407fqi30/AN2NIxPUrXvMEVjImsHmX2g?rlkey=qqxzkohhnmgjgz2abf6cvb32h&st=0kbpp7v4&dl=1",
    scenario_dir / "param.zip",
    param_dir
)

# Copy wireless params.mat file to wireless folder
wireless_dir = scenario_dir / "wireless"
shutil.copy(param_dir / "params.mat", wireless_dir / "params.mat")
shutil.copy(param_dir / "scenario1.csv", scenario_dir / "scenario1.csv")

print('Completed')


## DeepVerse dataset generation

In [ ]:
# path to the configuration file
config_path = f'scenarios/{scenario_name}/param/config.m'

# initialize ParameterManager and load parameters
param_manager = ParameterManager(config_path)
param_manager.params["scenes"] = list(range(10)) # 2411
param_manager.params["radar"]["enable"] = True
param_manager.params["comm"]["enable"] = True
param_manager.params["camera"] = False
param_manager.params["lidar"] = False
param_manager.params["position"] = False

# generate a dataset
dataset = Dataset(param_manager)


## Data postprcessing

The generated DeepVerse dataset consists of:
- The communication channel between the basestation and the user
- The radar intermediate frequency (IF) signal at the base station

In the following blocks we apply postprocessing on this data:
- We apply beamforming (from a beam codebook) to the communication channel to get the communication beam power
- We process the radar IF signal to generate the range-angle map

We will then save the postprocessed data.

Following the postprocessing, we divide the full dataset to training and testing datasets.
Additionaly, we extract the index of the optimal communication beam (i.e. highest power).


### Simple beam-steering codebook

In [8]:
def construct_codebook():
    def beam_steering_codebook(angles, num_z, num_x):
        d = 0.5
        k_z = np.arange(num_z)
        k_x = np.arange(num_x)
        
        codebook = []
        
        for beam_idx in range(angles.shape[0]):
            z_angle = angles[beam_idx, 0]
            x_angle = angles[beam_idx, 1]
            bf_vector_z = np.exp(1j * 2 * np.pi * k_z * d * np.cos(np.radians(z_angle)))
            bf_vector_x = np.exp(1j * 2 * np.pi * k_x * d * np.cos(np.radians(x_angle)))
            bf_vector = np.outer(bf_vector_z, bf_vector_x).flatten()
            codebook.append(bf_vector)
        
        return np.stack(codebook, axis=0)
    # construct beam steering codebook
    num_angles = 64
    x_angles = np.linspace(0, 180, num_angles + 1)[1:]
    x_angles = np.flip(x_angles)
    z_angles = np.full(num_angles, 90)
    beam_angles = np.column_stack((z_angles, x_angles))
    return beam_steering_codebook(beam_angles, 1, 16)

### Radar range-anlge map processing

In [9]:
def range_angle_map(data, fft_size = 64):
    data = torch.from_numpy(data)
    data = torch.fft.fft(data, axis=1) # Range FFT
    data -= torch.mean(data, axis=2, keepdims=True) # Static removal
    data = torch.fft.fft(data, n=fft_size, axis=0) # Angle FFT
    data = torch.abs(data).sum(axis=2, keepdim=True) # Sum over velocity
    data = data.numpy().transpose([2,1,0])
    return data*100.

### Save communication beam power and radar range-angle map

In [10]:
# apply codebook to bs-ue comm channel
codebook = construct_codebook()
beam_power = []
num_scene = len(dataset.params['scenes'])
for i in range(num_scene):
    channel = dataset.get_sample('comm-ue', index=i, bs_idx=0, ue_idx=0).coeffs
    beam_power_ = (np.abs(codebook @ np.squeeze(channel, 0))**2).sum(-1)
    beam_power.append(beam_power_)

# save data
os.makedirs(f'scenarios/{scenario_name}/radar', exist_ok=True)
os.makedirs(f'scenarios/{scenario_name}/power', exist_ok=True)
for i in range(num_scene):
    # get beam power
    ue_channel = dataset.get_sample('comm-ue', index=i, bs_idx=0, ue_idx=0).coeffs
    beam_power = (np.abs(codebook @ np.squeeze(ue_channel, 0))**2).sum(-1)
    np.savetxt(f'scenarios/{scenario_name}/power/{i}.txt', beam_power)

    # get radar IF
    radar_IF = dataset.get_sample('radar', index=i, bs_idx=0, ue_idx=0).coeffs.astype(np.complex64).squeeze(1)
    radar_ra = range_angle_map(radar_IF, fft_size=64)
    np.save(f'scenarios/{scenario_name}/radar/{i}.npy', radar_ra)

### Create training and testing dataset

In [11]:
csv_path = Path(f"scenarios/{scenario_name}/scenario1.csv")
csv_dir = csv_path.parent

# Load the CSV with index and seq_index
df = pd.read_csv(csv_path)  # Replace with your actual file path

# Keep only index and seq_index, and rename index -> scene
df = df[['index', 'seq_index']].rename(columns={'index': 'scene'})

# Generate the power and radar columns
df['power'] = df.index.map(lambda i: f'./power/{i}.txt')
df['radar'] = df.index.map(lambda i: f'./radar/{i}.npy')

# Save or inspect
df.to_csv(csv_dir / f'{scenario_name}.csv', index=False)

# Compute beam_index from power file
def get_beam_index(rel_path):
    try:
        abs_path = csv_dir / rel_path.strip()
        arr = np.loadtxt(abs_path)
        return int(np.argmax(arr))
    except Exception as e:
        print(f"Error reading {rel_path}: {e}")
        return -1

df['beam'] = df['power'].apply(get_beam_index)

# Split without data leakage based on seq_index
unique_seq_indices = df['seq_index'].unique()
train_seqs, test_seqs = train_test_split(unique_seq_indices, test_size=0.2, random_state=42)

train_df = df[df['seq_index'].isin(train_seqs)]
test_df = df[df['seq_index'].isin(test_seqs)]

# Save the split CSVs
train_df.to_csv(csv_dir / f'{scenario_name}_train.csv', index=False)
test_df.to_csv(csv_dir / f'{scenario_name}_test.csv', index=False)

## Deep learning demo

In this block we train a neural network for radar-aided beam prediction
- The input of the neural network is the radar range-angle map
- The ground-truth output of the neural network is a one-hot vector for the optimal beam index
- The loss function for training is the cross-entropy
- We evaluate the performance of the beam prediction with top-k accuracy

### Deep learning model

In [ ]:
class LeNet_RangeAngle(nn.Module):
    def __init__(self):
        super(LeNet_RangeAngle, self).__init__()
        self.pool = nn.AvgPool2d((2, 2))
        
        self.conv1 = nn.Conv2d(1, 8, 3, padding='same')
        self.conv2 = nn.Conv2d(8, 16, 3, padding='same')
        self.conv3 = nn.Conv2d(16, 8, 3, padding='same')
        self.conv4 = nn.Conv2d(8, 4, 3, padding='same')
        self.conv5 = nn.Conv2d(4, 2, 3, padding='same')
        self.fc1 = nn.Linear(1024, 4*64)
        self.fc2 = nn.Linear(4*64, 2*64)
        self.fc3 = nn.Linear(2*64, 64)
        self.name = 'LeNet_RangeAngle'

    def forward(self, x):
        
        x = F.relu((self.conv1(x)))
        x = F.relu((self.conv2(x)))
        x = F.relu(self.pool(self.conv3(x)))
        x = F.relu(self.pool(self.conv4(x)))
        x = F.relu(self.pool(self.conv5(x)))
        x = torch.flatten(x, start_dim=1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

### Training utilities

In [12]:
def load_dataset(root_dir, csv_file, radar_column='radar', label_column='beam'):
    csv_file = pd.read_csv(os.path.join(root_dir, csv_file))
    X = np.load(os.path.abspath(root_dir +  csv_file[radar_column][0]), allow_pickle=True)
    X = np.zeros((len(csv_file),) + X.shape, dtype=X.dtype)
    for i in tqdm(range(len(csv_file))):
        X[i] = np.load(os.path.abspath(root_dir +  csv_file[radar_column][i]), allow_pickle=True)
    y = np.array(csv_file[label_column])
    return X, y

def train_loop(X_train, y_train, net, optimizer, criterion, device, batch_size=64):
    net.train()
        
    running_acc = 0.0
    running_loss = 0.0
    running_size = 0
    with tqdm(iterate_minibatches(X_train, y_train, batch_size, shuffle=True), unit=' batch', 
              total=int(np.ceil(X_train.shape[0]/batch_size)), file=sys.stdout, leave=True) as tepoch:
        for batch_x, batch_y in tepoch:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            
            optimizer.zero_grad() # Make the gradients zero
            batch_y_hat = net(batch_x) # Prediction
            loss = criterion(batch_y_hat, batch_y) # Loss computation
            loss.backward() # Backward step
            optimizer.step() # Update coefficients
            
            predictions = batch_y_hat.argmax(dim=1, keepdim=True).squeeze()
            
            running_acc += (predictions == batch_y).sum().item()
            running_loss += loss.item() * batch_x.shape[0]
            running_size += batch_x.shape[0]
            curr_acc = 100. * running_acc/running_size
            curr_loss = running_loss/running_size

            
            tepoch.set_postfix(loss=curr_loss, accuracy=curr_acc)
            
        curr_acc = 100. * running_acc/len(X_train)
        curr_loss = running_loss/np.ceil(X_train.shape[0]/batch_size)

    return curr_loss, curr_acc
    
def eval_loop(X_val, y_val, net, criterion, device, batch_size=64):
    net.eval()
    
    running_acc = 0.0
    running_loss = 0.0
    with torch.no_grad():
        for batch_x, batch_y in iterate_minibatches(X_val, y_val, batch_size, shuffle=False):
            
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            
            batch_y_hat = net(batch_x) # Prediction
    
            predictions = batch_y_hat.argmax(dim=1, keepdim=True).squeeze()
            running_acc += (predictions == batch_y).sum().item()
            running_loss += criterion(batch_y_hat, batch_y).item() * batch_x.shape[0]
        
    
    curr_acc = 100. * running_acc/len(X_val)
    curr_loss = running_loss/len(X_val)
    print('Validation: [accuracy=%.2f, loss=%.4f]' % (curr_acc, curr_loss), flush=True)
    
    return curr_loss, curr_acc
    
def test_loop(X_test, y_test, net, device, model_path=None):
    # Network Setup
    if model_path is not None:
        net.load_state_dict(torch.load(model_path))
    net.eval()

    data_shape_single = list(X_test.shape)
    data_shape_single[0] = 1
    data_shape_single = tuple(data_shape_single)
    dummy_input = torch.randn(data_shape_single, dtype=torch.float, device=device)
    out = net(dummy_input)
    
    y = -1*torch.ones(len(X_test))
    y_hat = -1*torch.ones((len(X_test), out.shape[1])) #Top 5
    
    cur_ind = 0
    
    # Test
    with torch.no_grad():
        for batch_x, batch_y in iterate_minibatches(X_test, y_test, batchsize=1, shuffle=False):
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            batch_y_hat = net(batch_x) # Prediction

            next_ind = cur_ind + batch_x.shape[0]
            y[cur_ind:next_ind] = batch_y
            y_hat[cur_ind:next_ind, :] = batch_y_hat
            cur_ind = next_ind

    
    return y, y_hat
    

def evaluate_predictions(y, y_hat, k):
    topk_pred = torch.topk(y_hat, k=k).indices
    topk_acc = np.zeros(k)
    for i in range(k):
        topk_acc[i] = torch.mean((y == topk_pred[:, i])*1.0)
    topk_acc = np.cumsum(topk_acc)
    
    beam_dist = torch.mean(torch.abs(y - topk_pred[:, 0]))
    
    return topk_acc, beam_dist

def iterate_minibatches(X, y, batchsize, shuffle=False):
    
    data_len = X.shape[0]
    indices = np.arange(data_len)
    if shuffle:
        np.random.shuffle(indices)
        
    for start_idx in range(0, data_len, batchsize):
        end_idx = min(start_idx + batchsize, data_len)
        excerpt = indices[start_idx:end_idx]
        yield X[excerpt], y[excerpt]     

### Train model

In [ ]:
#############################
######## Load dataset #######
#############################

# dataset files
data_root_dir = f'scenarios/{scenario_name}'
train_csv = f'{scenario_name}_train.csv'
test_csv = f'{scenario_name}_test.csv'

# load and preprocess data
print('Preparing training dataset')
X_train, y_train = load_dataset(data_root_dir,train_csv)
print('Preparing test dataset')
X_test, y_test = load_dataset(data_root_dir, test_csv)
X_train = torch.from_numpy(X_train)
X_test = torch.from_numpy(X_test)
y_train = torch.from_numpy(y_train)
y_test = torch.from_numpy(y_test)


##########################
######## Training ########
##########################

# training hyperparameters
batch_size = 32
num_epoch = 40
learning_rate = 1e-3

# ckpts save dir
ckpt_dir = os.path.abspath(f'./checkpoints')
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)
print('Model ckpt path: %s' % ckpt_dir)

# reproducibility
torch.manual_seed(1115)
np.random.seed(1115)
torch.backends.cudnn.deterministic = True

# pytorch device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# init model
net = LeNet_RangeAngle().to(device)

# optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

# epochs
print('Training')
for epoch in range(num_epoch):
    print('Epoch %i/%i:'%(epoch+1, num_epoch), flush=True)
    train_loop(X_train, y_train, net, optimizer, criterion, device, batch_size=batch_size)
    eval_loop(X_test, y_test, net, criterion, device, batch_size=batch_size)
    scheduler.step()

torch.save(net.state_dict(), os.path.join(ckpt_dir, f'{net.name}.pth'))




### Test model

In [ ]:
print('Testing')
topk = 5
y_final, y_hat_final = test_loop(X_test, y_test, net, device, model_path=os.path.join(ckpt_dir, f'{net.name}.pth'))
topk_acc_final, beam_dist_final = evaluate_predictions(y_final, y_hat_final, k=topk)
print('Top-k Accuracy: ' + ' - '.join(['%.2f' for _ in range(topk)]) % tuple(topk_acc_final*100))